# Fourier Neural Operator 1D

## Burgers equation

---

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from utilities3 import *

In [3]:
torch.manual_seed(0)
np.random.seed(0)


In [6]:
import os
from pprint import pprint

from fourier_1d_for_nb import SpectralConv1d, FNO1d


# Hyperparamètres

In [7]:
################################################################
#  configurations
################################################################
ntrain = 1000
ntest = 100

sub = 2**3 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate
s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

modes = 16
width = 64

## Loading Data

In [18]:
################################################################
# read data
################################################################

# Data is of the shape (number of samples, grid size)
dataloader = MatReader('data/burgers_data_R10.mat')
x_data = dataloader.read_field('a')[:,::sub]
y_data = dataloader.read_field('u')[:,::sub]

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]


In [19]:
x_train.shape

torch.Size([1000, 1024])

In [20]:
x_train

tensor([[ 0.8354,  0.8348,  0.8340,  ...,  0.8364,  0.8362,  0.8359],
        [ 0.4741,  0.4723,  0.4707,  ...,  0.4804,  0.4781,  0.4760],
        [-0.3802, -0.3849, -0.3898,  ..., -0.3669, -0.3711, -0.3756],
        ...,
        [ 0.0938,  0.0969,  0.1002,  ...,  0.0856,  0.0882,  0.0909],
        [ 0.4329,  0.4335,  0.4340,  ...,  0.4302,  0.4312,  0.4321],
        [-0.5274, -0.5289, -0.5305,  ..., -0.5228, -0.5243, -0.5259]])

In [ ]:

# cat the locations information
grid = np.linspace(0, 1, s).reshape(1, s, 1)
grid = torch.tensor(grid, dtype=torch.float)
x_train = torch.cat([x_train.reshape(ntrain,s,1), grid.repeat(ntrain,1,1)], dim=2)
x_test = torch.cat([x_test.reshape(ntest,s,1), grid.repeat(ntest,1,1)], dim=2)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [16]:
grid

tensor([[[0.0000e+00],
         [9.7752e-04],
         [1.9550e-03],
         ...,
         [9.9804e-01],
         [9.9902e-01],
         [1.0000e+00]]])

In [21]:
grid.shape

torch.Size([1, 1024, 1])

In [11]:
x_data.shape

torch.Size([2048, 1024])

In [12]:
y_data.shape

torch.Size([2048, 1024])

In [13]:
x_train.shape

torch.Size([1000, 1024, 2])

In [14]:
x_test.shape

torch.Size([100, 1024, 2])

In [22]:
y_train.shape

torch.Size([1000, 1024])

In [23]:
y_test.shape

torch.Size([100, 1024])

In [ ]:
# model
model = FNO1d(modes, width).cuda()
print(count_params(model))